In [1]:
import os
import pandas as pd
import numpy as np
import time
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
from selenium.webdriver.common.by import By

## 1) Tor-Selenium Setup

In [7]:
torexe = os.popen(r'C:\Users\admin\Desktop\Tor Browser\Browser\TorBrowser\Tor\tor.exe')

firefoxBinary = r"C:\Users\admin\Desktop\Tor Browser\Browser\firefox.exe"
geckodriverPath = r"C:\Users\admin\Desktop\Tor Browser\Browser\geckodriver.exe"
proxyIP = "127.0.0.1"
proxyPort = 9150

binary = FirefoxBinary(firefoxBinary)

proxy_settings = {
    "network.proxy.type": 1,
    "network.proxy.socks": proxyIP,
    "network.proxy.socks_port": proxyPort,
    "network.proxy.socks_remote_dns": False,
    "extensions.torlauncher.start_tor": True
}

profile = FirefoxProfile(r'C:\Users\admin\Desktop\Tor Browser\Browser\TorBrowser\Data\Browser\profile.default')
profile.set_preference('extensions.torlauncher.start_tor', True) # bypass tor connection page
profile.set_preference('network.proxy.type', 1)
profile.set_preference('network.proxy.socks', '127.0.0.1')
profile.set_preference('network.proxy.socks_port', 9150)
profile.set_preference("network.proxy.socks_remote_dns", True)
profile.update_preferences()

driver = webdriver.Firefox(executable_path=geckodriverPath, 
                           firefox_binary=binary, 
                           firefox_profile=profile)



In [8]:
# Wait till your tor browser is connected to the tor network before executing this
driver.get('http://basemmnnqwxevlymli5bs36o5ynti55xojzvn246spahniugwkff2pad.onion/')


In [18]:

time.sleep(20)
    
SCROLL_PAUSE_TIME = 10 # as the Tor browser is slow, might need to increase this accordingly

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [45]:
companyNames = []
companyDescription = []
companyLocation = []
noClicks = []
dateVictimised = []
comment = []


cards = driver.find_elements_by_xpath("//div[@class='list-group-item rounded-3 py-3 bg-body-secondary text-bg-dark mb-2 position-relative']")

for card in cards:

    companyNameElement = card.find_element_by_xpath(".//a[@class='stretched-link']")
    companyNameText = companyNameElement.text
    companyNames.append(companyNameText)
    
    
    dateVictimisedElement = card.find_element_by_xpath("(.//div[@class='d-flex gap-2 small mt-1 opacity-25']/div)[2]")
    dateVictimisedText = dateVictimisedElement.text
    
    date_index = 8
    date_text = dateVictimisedText[date_index::]
    dateVictimised.append(date_text)
    
    noClicksElement = card.find_element_by_xpath("(.//div[@class='d-flex gap-2 small mt-1 opacity-25']/div)[3]")
    noClicksText = noClicksElement.text
    clicks_index = 6
    click_text = noClicksText[clicks_index::]
    noClicks.append(click_text)
    
    
    companyDescriptionElement = card.find_element_by_xpath(".//div[@class='small opacity-50']")
    companyDescriptionText = companyDescriptionElement.text
    companyDescription.append(companyDescriptionText)
    
    
    
    
    commentElement = card.find_element_by_xpath("(.//div[@class='small opacity-50'])[2]")
    commentText = commentElement.text
    
    comment_index = 30
    comment_text = commentText[comment_index::]
    new_text = comment_text.replace('\n',',')
    comment.append(new_text)
    
    
    
    

print(companyNames)
print(companyDescription)
print(companyLocation)
print(noClicks)
print(dateVictimised)
print(comment)
#driver.quit()

['KTUA Landscape Architecture and Planning', 'Comtek Advanced Structures, a Latecoere Company', 'ExdionInsurance', 'Petersen Johnson', 'Ted Pella Inc.', 'Sabian Inc', 'Chula Vista Electric (CVE)', 'C.F. Service and Supply', 'Notel', 'Muenz-Engineered Sales', 'SUD TRADING COMPANY', 'WEBBER RESTAURANT GROUP', 'Praxis Arndt und Langer', 'Springer Eubank', 'The Envelope Works Ltd', 'FabricATE Engineering', 'Araújo e Policastro Advogados', 'Ford Covesa', 'UNIVERSAL REALTY GROUP', 'Conselho Superior da Justiça do Trabalho', 'FRESH TASTE PRODUCE USA AND ASSOCIATES INC.', 'Prodegest Assessors', 'VVandA', 'Fenn Termite and Pest Control', 'Petkus Brothers', 'PRIDE GLOBAL CONSULTING SL', 'PT. Cahaya Benteng Mas', 'Shanghai FRP Research Institute Co., Ltd.', 'KLM Laboratories Pvt. Ltd', 'Varna Packaging', 'SKYROOT', 'Sydenham Laboratories', 'FA Foundry', 'INSTITUTO NACIONAL DE ELECTRIFICACION', 'HFH Capital', 'Bahamas Medical and Surgical Supplies', 'The Law Offices of Steven H. Heisler', 'Kevills

## 3) Clean data + Extract to CSV/Excel format


Requirements for data:  
- Date of victim company within 2023

In [46]:
df = pd.DataFrame({"Company": companyNames, "Company Description": companyDescription, "Date Victimised": dateVictimised, "No of views": noClicks, "Comment": comment})
df



Company  \
0           KTUA Landscape Architecture and Planning   
1    Comtek Advanced Structures, a Latecoere Company   
2                                    ExdionInsurance   
3                                   Petersen Johnson   
4                                     Ted Pella Inc.   
..                                               ...   
183                                Concept Fasteners   
184                                       MTS Office   
185                                           ESSPEE   
186  GIOTTO - COMÉRCIO DE VESTUÁRIO, UNIPESSOAL, LDA   
187                            P1 Technical Services   

                                   Company Description Date Victimised  \
0    As planners and landscape architects, KTUA is ...      17.10.2023   
1    Comtek designs, manufactures and repairs compo...      17.10.2023   
2    Exdion is the leading Digital Insurance Platfo...      17.10.2023   
3    Petersen Johnson is a leading Arizona personal...      15.10.2023   
4    Electron microscopy supplies and instruments, ...      10.10.2023   
..                                                 ...             ...   
183  Concept Fasteners are dedicated to manufacturi...      06.06.2022   
184  MTS Office is the authorized Toshiba and Lexma...      15.05.2022   
185  Market leaders in the field of resistant mater...      05.03.2022   
186  GIOTTO - Comércio de Vestuário Unipessoal, Lda...      26.04.2022   
187  http://p1-tech.com/\n\nP1 Technical Services h...      16.04.2022   

    No of views                                            Comment  
0           919  Invoice,Receipts,Accounting documents,Personal...  
1           906  Invoice,Receipts,Accounting documents,Personal...  
2          2086  Invoice,Receipts,Accounting documents,Personal...  
3          3365  Invoice,Receipts,Accounting documents,Personal...  
4          6548  Invoice,Receipts,Accounting documents,Personal...  
..          ...                                                ...  
183       79141                           S#knzaQ2bIocBsRInP60n_XA  
184       77688                                      a access!!!!!  
185       77169   of this company and we are ready to share. wr...  
186       78241  irectory/company-profiles.GIOTTO_-_COM%C3%89RC...  
187       77112  5ER,,UPD: Do you need an up to date link? writ...  

[188 rows x 5 columns]

In [47]:
df.to_csv("8BASE.csv")

## 3) Scraping Glassdoor for more company information

More information we need:  
- Industry  
- Geography  
  - Country  

Additional information we can scrape (the more we have, the more insights we can infer):
- Company Website

In [44]:
# pip install webdriver-manager
import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager


df = pd.read_csv('8BASE.csv')

In [45]:
driver_path = GeckoDriverManager().install()

# Specify the path using executable_path
driver = webdriver.Firefox(executable_path=driver_path)
driver.maximize_window()

/var/folders/_p/rjp8kjjd29j3f3m7h2cyl20r0000gn/T/ipykernel_5798/3726700424.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=driver_path)


In [46]:
unique_dates = df['Date Victimised'].unique()
print(unique_dates)


[' 17.10.2023' ' 15.10.2023' ' 10.10.2023' ' 06.10.2023' ' 05.10.2023'
 ' 04.10.2023' ' 03.10.2023' ' 02.10.2023' ' 30.09.2023' ' 25.09.2023'
 ' 18.09.2023' ' 15.09.2023' ' 14.09.2023' ' 13.09.2023' ' 11.09.2023'
 ' 07.09.2023' ' 06.09.2023' ' 05.09.2023' ' 03.09.2023' ' 02.09.2023'
 ' 01.09.2023' ' 31.08.2023' ' 23.08.2023' ' 22.08.2023' ' 11.08.2023'
 ' 13.08.2023' ' 10.08.2023' ' 08.08.2023' ' 12.08.2023' ' 01.08.2023'
 ' 31.07.2023' ' 30.07.2023' ' 29.07.2023' ' 22.07.2023' ' 18.07.2023'
 ' 20.07.2023' ' 21.07.2023' ' 17.07.23' ' 18.07.23' ' 15.07.23'
 ' 14.07.23' ' 13.07.2023' ' 11.07.2023' ' 10.07.2023' ' 03.07.2023'
 ' 26.06.2023' ' 25.06.2023' ' 22.06.2023' ' 21.06.2023' ' 19.06.2023'
 ' 11.06.2023' ' 14.05.2023' ' 1.06.2023' ' 09.05.2023' ' 29.04.2023'
 ' 02.04.2023' ' 01.04.2023' ' 24.03.2023' ' 19.03.2023' ' 16.03.2023'
 ' 25.02.2023' ' 15.03.2023' ' 12.03.2023' ' 07.03.2023' ' 05.03.2023'
 ' 10.02.2023' ' 02.03.2023' ' 23.02.2023' ' 18.02.2023' ' 13.02.2023'
 ' 05.02.2023' 

In [47]:
# Trim leading spaces
df['Date Victimised'] = df['Date Victimised'].str.strip()

# Convert to datetime
def custom_date_parser(date_string):
    try:
        return pd.to_datetime(date_string, format="%d.%m.%Y")
    except ValueError:
        return pd.to_datetime(date_string, format="%d.%m.%y")

df['Date Victimised'] = df['Date Victimised'].apply(custom_date_parser)

# Filter the dataframe for only rows where the year is 2023 or later
df = df[df['Date Victimised'].dt.year >= 2023]


In [41]:
df

Unnamed: 0                                          Company  \
0             0         KTUA Landscape Architecture and Planning   
1             1  Comtek Advanced Structures, a Latecoere Company   
2             2                                  ExdionInsurance   
3             3                                 Petersen Johnson   
4             4                                   Ted Pella Inc.   
..          ...                                              ...   
147         147  China Export &amp; Credit Insurance Corporation   
148         148                                   Conklin Benham   
149         149                  FORMA ESPACOS IMOBILIARIOS LTDA   
150         150                           Ayers Mechanical Group   
170         170                                          Brandao   

                                   Company Description Date Victimised  \
0    As planners and landscape architects, KTUA is ...      2023-10-17   
1    Comtek designs, manufactures and repairs compo...      2023-10-17   
2    Exdion is the leading Digital Insurance Platfo...      2023-10-17   
3    Petersen Johnson is a leading Arizona personal...      2023-10-15   
4    Electron microscopy supplies and instruments, ...      2023-10-10   
..                                                 ...             ...   
147  China Export & Credit Insurance Corporation (h...      2023-01-24   
148  For well over a half-century, Conklin Benham h...      2023-01-24   
149  FORMA ESPAÇOS IMOBILIARIOS, a company with 16 ...      2023-01-14   
150  Ayers Mechanical Group has been providing fast...      2023-01-10   
170  Brandão e Endo is an Accounting company founde...      2023-09-06   

     No of views                                            Comment  
0            919  Invoice,Receipts,Accounting documents,Personal...  
1            906  Invoice,Receipts,Accounting documents,Personal...  
2           2086  Invoice,Receipts,Accounting documents,Personal...  
3           3365  Invoice,Receipts,Accounting documents,Personal...  
4           6548  Invoice,Receipts,Accounting documents,Personal...  
..           ...                                                ...  
147        83813                           Q#r4jDhKwwYN5afPRM7Oex4g  
148        81887                           C#VaNnz6oBCQ268SkQ3QsFSg  
149        77917  nz/#P!AgAhAgTNEAi8mEGkb1FprOZVBzMoLRVQbhDPGsiK...  
150        81468                           T#f-Mn7x8UfmaRgfBBrswMqg  
170        76505                           S#xtBjkHvjb822X8HiJahbDg  

[152 rows x 6 columns]

In [48]:
import requests

def location_to_country(location):
    base_url = "https://nominatim.openstreetmap.org/search"
    
    params = {
        "addressdetails": 1,
        "q": location,
        "format": "jsonv2",
        "limit": 1
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    if data:
        country = data[0].get("address", {}).get("country", "")
        return country

    return np.nan

In [49]:
companyWebsite_list = []
industry_list = []
headquarter_list = []
country_list = []
number = 0

wait = WebDriverWait(driver, 10)

for company in df['Company']:
    try:
        #print(f"Searching for {company}")
        driver.get('https://www.google.com')
        search = driver.find_element(By.NAME, 'q')
        search.clear()  # Clear any previous input
        search.send_keys(company + ' linkedin')
        #print(f"Sent search for {company}")
        wait.until(EC.element_to_be_clickable((By.NAME, 'q')))
        search.send_keys(Keys.RETURN)
        #print(f"Clicked RETURN for {company}")
        wait.until(EC.presence_of_element_located((By.TAG_NAME, 'h3')))
        linkedin_page = driver.find_element_by_tag_name('h3')  # clicking the first search result
        time.sleep(2)  # to limit rate of requests to linkedin
        linkedin_page.click()
        #print(f"Clicked on search result for {company}")
        if "linkedin" in driver.current_url:

            loaded = wait.until(EC.presence_of_element_located((By.ID, "main-content")))

            # Check if the scraped company name is even in the linkedin page (sometimes the company data is not in linkedin; false positive)
            # try:
            #     linkedin_companyName_element = driver.find_element_by_xpath("//h1[@class='top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0']")
            #     linkedin_companyName = linkedin_companyName_element.text
            # except:
            #     print(company)
            #     continue

            # if not substring_included(company, linkedin_companyName):
            #     print(company)
            #     continue

            # bypass sign in modal
            try:
                sign_in_button_element = driver.find_element_by_xpath("//button[@class='sign-in-modal__outlet-btn cursor-pointer btn-md btn-primary']")
                sign_in_button_element.click()
                dismiss_button_element = driver.find_element_by_xpath("//button[@class='modal__dismiss btn-tertiary h-[40px] w-[40px] p-0 rounded-full indent-0 sign-in-modal__dismiss absolute right-0 cursor-pointer m-[20px]']")
                dismiss_button_element.click()
            except:
                pass

            CLASS = ".//dd[@class='font-sans text-md text-color-text break-words overflow-hidden']"

            try:
                website_element = driver.find_element_by_css_selector('[data-test-id="about-us__website"]')
                website_text_element = website_element.find_element_by_xpath(CLASS)
                website_text = website_text_element.text
            except Exception as e:
                website_text = np.nan
                print(e)
            companyWebsite_list.append(website_text)

            try:
                industry_element = driver.find_element_by_css_selector('[data-test-id="about-us__industry"]')
                industry_text_element = industry_element.find_element_by_xpath(CLASS)
                industry_text = industry_text_element.text
            except Exception as e:
                industry_text = np.nan
                print(e)
            industry_list.append(industry_text)
            
            try:
                headquarters_element = driver.find_element_by_css_selector('[data-test-id="about-us__headquarters"]')
                headquarters_text_element = headquarters_element.find_element_by_xpath(CLASS)
                headquarters_text = headquarters_text_element.text

                country = location_to_country(headquarters_text)
            except Exception as e:
                headquarters_text = np.nan
                country = np.nan
                print(e)
            headquarter_list.append(headquarters_text)
            country_list.append(country)

        else:
            companyWebsite_list.append(np.nan)
            industry_list.append(np.nan)
            headquarter_list.append(np.nan)
            country_list.append(np.nan)

    except Exception as e:
        companyWebsite_list.append(np.nan)
        industry_list.append(np.nan)
        headquarter_list.append(np.nan)
        country_list.append(np.nan)
        print(e)
    number += 1
    
driver.quit()


/var/folders/_p/rjp8kjjd29j3f3m7h2cyl20r0000gn/T/ipykernel_5798/919396487.py:21: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  linkedin_page = driver.find_element_by_tag_name('h3')  # clicking the first search result
/var/folders/_p/rjp8kjjd29j3f3m7h2cyl20r0000gn/T/ipykernel_5798/919396487.py:43: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  sign_in_button_element = driver.find_element_by_xpath("//button[@class='sign-in-modal__outlet-btn cursor-pointer btn-md btn-primary']")
/var/folders/_p/rjp8kjjd29j3f3m7h2cyl20r0000gn/T/ipykernel_5798/919396487.py:53: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  website_element = driver.find_element_by_css_selector('[data-test-id="about-us__website"]')
/Users/hudeyuh/Library/Python/3.9/lib/python/site-packages/selenium/webdriver/remote/webelement.py:392: UserWarning: find_element_by_* command

Message: Unable to locate element: [data-test-id="about-us__website"]
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16

Message: Unable to locate element: [data-test-id="about-us__industry"]
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16

Message: Unable to locate element: [data-test-id="about-us__headquarters"]
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome:

/var/folders/_p/rjp8kjjd29j3f3m7h2cyl20r0000gn/T/ipykernel_5798/919396487.py:45: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  dismiss_button_element = driver.find_element_by_xpath("//button[@class='modal__dismiss btn-tertiary h-[40px] w-[40px] p-0 rounded-full indent-0 sign-in-modal__dismiss absolute right-0 cursor-pointer m-[20px]']")


Message: Unable to locate element: [data-test-id="about-us__website"]
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16

Message: Unable to locate element: [data-test-id="about-us__industry"]
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16

Message: Unable to locate element: [data-test-id="about-us__headquarters"]
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome:

In [50]:
df["Company Website"] = companyWebsite_list
df["Industry"] = industry_list
df["Location"] = headquarter_list
df["Country"] = country_list
df

Unnamed: 0                                          Company  \
0             0         KTUA Landscape Architecture and Planning   
1             1  Comtek Advanced Structures, a Latecoere Company   
2             2                                  ExdionInsurance   
3             3                                 Petersen Johnson   
4             4                                   Ted Pella Inc.   
..          ...                                              ...   
147         147  China Export &amp; Credit Insurance Corporation   
148         148                                   Conklin Benham   
149         149                  FORMA ESPACOS IMOBILIARIOS LTDA   
150         150                           Ayers Mechanical Group   
170         170                                          Brandao   

                                   Company Description Date Victimised  \
0    As planners and landscape architects, KTUA is ...      2023-10-17   
1    Comtek designs, manufactures and repairs compo...      2023-10-17   
2    Exdion is the leading Digital Insurance Platfo...      2023-10-17   
3    Petersen Johnson is a leading Arizona personal...      2023-10-15   
4    Electron microscopy supplies and instruments, ...      2023-10-10   
..                                                 ...             ...   
147  China Export & Credit Insurance Corporation (h...      2023-01-24   
148  For well over a half-century, Conklin Benham h...      2023-01-24   
149  FORMA ESPAÇOS IMOBILIARIOS, a company with 16 ...      2023-01-14   
150  Ayers Mechanical Group has been providing fast...      2023-01-10   
170  Brandão e Endo is an Accounting company founde...      2023-09-06   

     No of views                                            Comment  \
0            919  Invoice,Receipts,Accounting documents,Personal...   
1            906  Invoice,Receipts,Accounting documents,Personal...   
2           2086  Invoice,Receipts,Accounting documents,Personal...   
3           3365  Invoice,Receipts,Accounting documents,Personal...   
4           6548  Invoice,Receipts,Accounting documents,Personal...   
..           ...                                                ...   
147        83813                           Q#r4jDhKwwYN5afPRM7Oex4g   
148        81887                           C#VaNnz6oBCQ268SkQ3QsFSg   
149        77917  nz/#P!AgAhAgTNEAi8mEGkb1FprOZVBzMoLRVQbhDPGsiK...   
150        81468                           T#f-Mn7x8UfmaRgfBBrswMqg   
170        76505                           S#xtBjkHvjb822X8HiJahbDg   

                      Company Website  \
0                 http://www.ktua.com   
1       http://www.comtekadvanced.com   
2    https://www.exdioninsurance.com/   
3      http://www.petersonjohnson.com   
4             http://www.tedpella.com   
..                                ...   
147        http://www.sinosure.com.cn   
148          http://conklinbenham.com   
149       http://viveronebento.com.br   
150   http://www.ayersmechanical.com/   
170                               NaN   

                                           Industry  \
0                         Architecture and Planning   
1    Aviation and Aerospace Component Manufacturing   
2                  Business Consulting and Services   
3                                      Law Practice   
4                           Nanotechnology Research   
..                                              ...   
147                              Financial Services   
148                                    Law Practice   
149                                     Real Estate   
150                                    Construction   
170                                             NaN   

                               Location        Country  
0                         San Diego, CA  United States  
1                   Burlington, Ontario         Canada  
2                          Plano, Texas  United States  
3                      Phoenix, Arizona  United States  
4      

In [51]:
df.to_csv("8BASE_Linkedin_Data.csv")